# <font color="#418FDE" size="6.5" uppercase>**super And Methods**</font>

>Last update: 20251221.
    
By the end of this Lecture, you will be able to:
- Use super to call parent class implementations correctly in single and multiple inheritance scenarios. 
- Define and use classmethod and staticmethod to represent behaviors tied to classes rather than instances. 
- Create and use memoryview objects to share slices of binary data without copying. 


## **1. Mastering super Calls**

### **1.1. Basic super usage**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_07/Lecture_B/image_01_01.jpg?v=1766324327" width="250">



>* Child classes extend, not replace, parent behavior
>* Delegation construct calls parent method on current object

>* super binds to class and instance together
>* lets child methods reuse parent behavior, then extend

>* Using super avoids brittle, hard-coded parent references
>* It keeps changing class hierarchies working safely



In [ ]:
#@title Python Code - Basic super usage

# Demonstrate basic super usage with simple order classes.
# Show how child classes extend parent behavior cleanly.
# Compare direct parent calls with flexible super based delegation.

class Order:
    def __init__(self, base_price_dollars):
        self.base_price_dollars = base_price_dollars

    def total_price(self):
        print("Order.total_price running for base order.")
        return self.base_price_dollars


class InternationalOrderBad(Order):
    def total_price(self):
        print("InternationalOrderBad.total_price running now.")
        base_total = Order.total_price(self)
        return base_total + 15


class InternationalOrderGood(Order):
    def total_price(self):
        print("InternationalOrderGood.total_price running now.")
        base_total = super().total_price()
        return base_total + 15


print("Creating base order with twenty dollars.")
base_order = Order(20)
print("Base order total price is:", base_order.total_price())


print("Creating bad international order now.")
intl_bad = InternationalOrderBad(20)
print("Bad international order total is:", intl_bad.total_price())


print("Creating good international order now.")
intl_good = InternationalOrderGood(20)
print("Good international order total is:", intl_good.total_price())



### **1.2. Understanding Method Resolution**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_07/Lecture_B/image_01_02.jpg?v=1766324376" width="250">



>* Python uses MRO to choose method implementations
>* super follows the MRO, not direct parents

>* MRO in single inheritance follows a simple chain
>* super walks this chain, avoiding hard-coded parent names

>* Multiple inheritance uses a single linear method order
>* Consistent super calls ensure cooperative, predictable behavior



In [ ]:
#@title Python Code - Understanding Method Resolution

# Demonstrate method resolution order using simple single inheritance classes.
# Show how super follows the method resolution order chain automatically.
# Print class names and MRO list to visualize resolution behavior.

class Vehicle:
    def describe(self):
        print("Vehicle describe called, base behavior here.")


class Car(Vehicle):
    def describe(self):
        print("Car describe called, before super call.")
        super().describe()


class SportsCar(Car):
    def describe(self):
        print("SportsCar describe called, before super call.")
        super().describe()


sports_car_instance = SportsCar()
print("Calling describe on SportsCar instance now.")
sports_car_instance.describe()


print("\nMethod resolution order for SportsCar class.")
for cls in SportsCar.mro():
    print("MRO entry:", cls.__name__)



### **1.3. Cooperative Multiple Inheritance**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_07/Lecture_B/image_01_03.jpg?v=1766324426" width="250">



>* Cooperative inheritance makes super essential for predictability
>* Each mixin calls super so all behaviors run

>* Let MRO and super choose next method
>* Mixins can be reordered or added without rewrites

>* Skipping super can silently bypass important steps
>* Consistent super use enables safe, modular mixins



In [ ]:
#@title Python Code - Cooperative Multiple Inheritance

# Demonstrate cooperative multiple inheritance using super across mixin classes.
# Show how each mixin participates without knowing full class hierarchy.
# Compare cooperative behavior with noncooperative direct parent calls.

class BaseRequest:
    def send(self, message):
        print("BaseRequest sending raw message.")
        return message


class AuthMixin:
    def send(self, message):
        print("AuthMixin adding simple authentication header.")
        message = f"AUTH|{message}"
        return super().send(message)


class CompressMixin:
    def send(self, message):
        print("CompressMixin simulating compression step.")
        message = f"COMPRESSED({message})"
        return super().send(message)


class BadEncryptMixin:
    def send(self, message):
        print("BadEncryptMixin encrypting but skipping cooperative super.")
        message = f"ENCRYPTED[{message}]"
        return BaseRequest.send(self, message)


class GoodEncryptMixin:
    def send(self, message):
        print("GoodEncryptMixin encrypting and calling cooperative super.")
        message = f"ENCRYPTED[{message}]"
        return super().send(message)


class BadClient(BadEncryptMixin, AuthMixin, CompressMixin, BaseRequest):
    pass


class GoodClient(GoodEncryptMixin, AuthMixin, CompressMixin, BaseRequest):
    pass


print("BadClient method resolution order sequence.")
print(BadClient.mro())
print("Sending using BadClient with noncooperative mixin.")
result_bad = BadClient().send("Hello from client.")
print("Final BadClient message result:", result_bad)


print("GoodClient method resolution order sequence.")
print(GoodClient.mro())
print("Sending using GoodClient with cooperative mixins.")
result_good = GoodClient().send("Hello from client.")
print("Final GoodClient message result:", result_good)



## **2. Class and Static Methods**

### **2.1. Classmethod Factory Patterns**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_07/Lecture_B/image_02_01.jpg?v=1766324479" width="250">



>* Class methods act as flexible factory constructors
>* They respect inheritance and separate intent from details

>* Different account creation paths become classmethod factories
>* Named factories parse inputs, reuse constructor, reduce duplication

>* Factories stay polymorphic across subclasses automatically
>* Centralized creation logic simplifies extension and maintenance



In [ ]:
#@title Python Code - Classmethod Factory Patterns

# Demonstrate classmethod factory patterns for creating account objects easily.
# Show how different factories create instances using the same underlying constructor.
# Highlight that subclasses automatically benefit from inherited classmethod factories.

class Account:
    def __init__(self, identifier, name, status):
        self.identifier = identifier
        self.name = name
        self.status = status

    @classmethod
    def from_legacy_record(cls, record_text):
        parts = record_text.strip().split(",")
        identifier = int(parts[0]) if parts[0] else 0
        name = parts[1] if parts[1] else "Unknown"
        status = parts[2] if len(parts) > 2 else "inactive"
        return cls(identifier, name, status)

    @classmethod
    def from_environment(cls):
        identifier = 999
        name = "Admin User"
        status = "active"
        return cls(identifier, name, status)

    def describe(self):
        return f"Account {self.identifier} named {self.name} is {self.status}."


class PremiumAccount(Account):
    def __init__(self, identifier, name, status):
        super().__init__(identifier, name, status)
        self.premium_support_hours = 24

    def describe(self):
        base = super().describe()
        return f"{base} Premium support {self.premium_support_hours} hours daily."


legacy_text = "101,Jane Doe,active"
regular_from_legacy = Account.from_legacy_record(legacy_text)

premium_from_legacy = PremiumAccount.from_legacy_record(legacy_text)

admin_from_environment = Account.from_environment()

print("Regular account type and description:")
print(type(regular_from_legacy), regular_from_legacy.describe())

print("Premium account type and description:")
print(type(premium_from_legacy), premium_from_legacy.describe())

print("Admin account type and description:")
print(type(admin_from_environment), admin_from_environment.describe())



### **2.2. Static Utility Methods**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_07/Lecture_B/image_02_02.jpg?v=1766324527" width="250">



>* Static methods are class-related, stateless helper functions
>* They improve organization, discoverability, and testability

>* Use static methods for logic not needing state
>* Keep related helper functions inside their concept class

>* Static methods are reusable, side-effect-free utilities
>* They clarify module boundaries and organize related functionality



In [ ]:
#@title Python Code - Static Utility Methods

# Demonstrate static utility methods inside a simple class example.
# Show that static methods ignore instance and class internal state.
# Group related utility functions inside a class for easier code navigation.

class DistanceTools:
    """Utility tools for simple distance related calculations."""

    def __init__(self, home_zip_code):
        self.home_zip_code = home_zip_code

    @staticmethod
    def miles_to_feet(miles_value):
        feet_per_mile = 5280
        return miles_value * feet_per_mile

    @staticmethod
    def estimate_taxi_cost(miles_value):
        base_fare_dollars = 3.00
        per_mile_rate_dollars = 2.50
        return base_fare_dollars + miles_value * per_mile_rate_dollars


trip_miles = 2.5

print("Trip distance in miles:", trip_miles)

print("Trip distance in feet:", DistanceTools.miles_to_feet(trip_miles))

print("Estimated taxi cost dollars:", DistanceTools.estimate_taxi_cost(trip_miles))

home_distance_tools = DistanceTools("10001")

print("Instance call feet:", home_distance_tools.miles_to_feet(trip_miles))

print("Instance call cost dollars:", home_distance_tools.estimate_taxi_cost(trip_miles))



### **2.3. Using cls and self**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_07/Lecture_B/image_02_03.jpg?v=1766324574" width="250">



>* self refers to one specific object instance
>* cls refers to the class blueprint and configuration

>* Instance methods use self for per-object state
>* Class methods use cls for shared data, construction

>* Choose instance, class, or static based on data
>* Clear roles make APIs reusable and maintainable



In [ ]:
#@title Python Code - Using cls and self

# Show difference between self and cls usage in methods.
# Demonstrate instance, class, and static method behaviors clearly.
# Keep example simple, readable, and beginner friendly overall.

class DiscountCalculator:
    base_discount_percent = 10

    def __init__(self, customer_name, purchase_amount):
        self.customer_name = customer_name
        self.purchase_amount = purchase_amount

    def apply_discount(self):
        discount_amount = self.purchase_amount * self.base_discount_percent / 100
        final_amount = self.purchase_amount - discount_amount

        print("Instance method using self attributes.")
        print("Customer:", self.customer_name, "Original:", self.purchase_amount, "Final:", final_amount)

    @classmethod
    def set_base_discount(cls, new_percent):
        print("Class method using cls attribute.")
        print("Old discount percent:", cls.base_discount_percent)
        cls.base_discount_percent = new_percent
        print("New discount percent:", cls.base_discount_percent)

    @staticmethod
    def dollars_to_cents(amount_dollars):
        cents_value = int(amount_dollars * 100)
        print("Static method without self or cls.")
        print("Dollars:", amount_dollars, "Cents:", cents_value)


DiscountCalculator.set_base_discount(15)

order = DiscountCalculator("Alice", 120.0)
order.apply_discount()

DiscountCalculator.dollars_to_cents(3.75)



## **3. Memoryview Essentials**

### **3.1. Bytes to Memoryview**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_07/Lecture_B/image_03_01.jpg?v=1766324638" width="250">



>* Memoryview is a window onto existing binary data
>* It avoids copying, improving performance on large datasets

>* Memoryview is a movable, shareable view onto bytes
>* View is read-only, enabling efficient, copy-free slicing

>* Server shares one bytes buffer via many views
>* Targeted views avoid copying, boosting speed and efficiency



In [ ]:
#@title Python Code - Bytes to Memoryview

# Demonstrate creating memoryview from bytes without copying data.
# Show that memoryview shares underlying immutable bytes buffer.
# Compare slicing bytes and slicing memoryview for efficiency understanding.

# Create an immutable bytes object representing simple binary data.
original_bytes = b"HEADER1234PAYLOAD5678"

# Create a memoryview that references the same underlying bytes buffer.
view = memoryview(original_bytes)

# Slice the original bytes to get header and payload copies.
header_bytes = original_bytes[:10]
payload_bytes = original_bytes[10:]

# Slice the memoryview to get header and payload views.
header_view = view[:10]
payload_view = view[10:]

# Print basic information showing that data content looks identical.
print("Original bytes content:", original_bytes)
print("Header bytes slice:", header_bytes)
print("Payload bytes slice:", payload_bytes)

# Show that memoryview slices expose bytes when converted explicitly.
print("Header view as bytes:", header_view.tobytes())
print("Payload view as bytes:", payload_view.tobytes())

# Show that memoryview reports read-only status for underlying immutable bytes.
print("Is original view read-only?", view.readonly)
print("Is header view read-only?", header_view.readonly)
print("Is payload view read-only?", payload_view.readonly)



### **3.2. Zero copy slicing**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_07/Lecture_B/image_03_02.jpg?v=1766324680" width="250">



>* Memoryview lets you slice binary data without copying
>* Slices are lightweight windows onto the same buffer

>* Memoryview slices share one buffer, avoiding copies
>* Enables fast, low-memory processing of binary streams

>* Keep the original buffer alive and stable
>* Define ownership, mutability, and read-only slice rules



In [ ]:
#@title Python Code - Zero copy slicing

# Demonstrate memoryview zero copy slicing with shared underlying buffer behavior.
# Show that slices share data without creating new copied byte storage.
# Highlight how changes through one slice appear in overlapping slices.

# Create a mutable bytearray buffer representing simple sensor inches values.
buffer = bytearray([10, 20, 30, 40, 50, 60, 70, 80])

# Create a memoryview window over the entire underlying buffer bytes.
view = memoryview(buffer)

# Take two overlapping zero copy slices from the original memoryview.
front_slice = view[0:4]
back_slice = view[2:6]

# Show original buffer and both slices before any modifications occur.
print("Before change:", list(buffer), list(front_slice), list(back_slice))

# Modify one element through the back_slice memoryview window.
back_slice[1] = 99

# Show buffer and slices after modification to demonstrate shared storage.
print("After change:", list(buffer), list(front_slice), list(back_slice))

# Confirm that id values differ while underlying bytes storage remains shared.
print("Object ids:", id(buffer), id(view), id(front_slice), id(back_slice))



### **3.3. Editing Bytearrays Safely**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_07/Lecture_B/image_03_03.jpg?v=1766324733" width="250">



>* Memoryview edits directly change the original bytearray
>* Plan precise changes to avoid corrupting shared data

>* Match replacement bytes to slice size exactly
>* Validate lengths, formats, and document byte positions

>* Coordinate shared memoryviews to avoid conflicting edits
>* Keep underlying bytearray stable while views exist



In [ ]:
#@title Python Code - Editing Bytearrays Safely

# Demonstrate safe memoryview editing on bytearrays without unnecessary copying.
# Show how edits affect both memoryview and original bytearray immediately.
# Emphasize careful sizing, positions, and shared buffer lifetime safety.

packet = bytearray(b"HDR1TEMP72FEND")  # Example packet with simple header and payload.
print("Original packet bytes:", packet)  # Show initial packet content for reference.

view_header = memoryview(packet)[0:4]  # Create header memoryview referencing first four bytes.
view_temp = memoryview(packet)[4:11]  # Create temperature memoryview referencing middle bytes.

print("Header before change:", bytes(view_header))  # Display header before modification.
print("Temp field before change:", bytes(view_temp))  # Display temperature field before modification.

new_header = b"HDR2"  # Prepare new header bytes with exactly matching length.
if len(new_header) == len(view_header):  # Validate replacement size matches header view length.
    view_header[:] = new_header  # Safely overwrite header bytes through memoryview slice.

new_temp = b"TEMP75F"  # Prepare new temperature bytes with correct seven byte length.
if len(new_temp) == len(view_temp):  # Validate replacement size matches temperature view length.
    view_temp[:] = new_temp  # Safely overwrite temperature bytes through memoryview slice.

print("Header after change:", bytes(view_header))  # Confirm header update through memoryview.
print("Temp field after change:", bytes(view_temp))  # Confirm temperature update through memoryview.

print("Packet after safe edits:", packet)  # Show entire packet reflecting both safe edits.




# <font color="#418FDE" size="6.5" uppercase>**super And Methods**</font>


In this lecture, you learned to:
- Use super to call parent class implementations correctly in single and multiple inheritance scenarios. 
- Define and use classmethod and staticmethod to represent behaviors tied to classes rather than instances. 
- Create and use memoryview objects to share slices of binary data without copying. 

In the next Module (Module 8), we will go over 'Errors And Exceptions'